In [1]:
import pandas as pd
import numpy as np

In [2]:
fin_df = pd.read_csv('../dataset/final_finance_time_series.csv', skiprows = 8)
print(fin_df.columns)

Index(['Reference Code', 'Date Time', 'Description', 'Dr.', 'Cr.', 'Status',
       'Balance (NPR)', 'Channel', 'Category'],
      dtype='object')


In [3]:
print(fin_df['Category'].value_counts())

Category
Groceries & Shopping    458
Banking & Finance       427
Dining & Food           423
Income                  417
Subscriptions           410
Personal Care           402
Entertainment           400
Travel                  400
Education               400
Housing & Utilities       1
Name: count, dtype: int64


In [4]:
def cleaner_function(df):
    df.columns = df.columns.str.strip()
    df = df.dropna(subset = ["Date Time", "Dr.", "Cr.", "Category"])
    df["Date Time"] = pd.to_datetime(df["Date Time"])
    df.set_index("Date Time", inplace = True)
    df.sort_index(inplace = True)
    return df

In [5]:
fin_df = cleaner_function(fin_df)


In [6]:
print(fin_df.columns)

Index(['Reference Code', 'Description', 'Dr.', 'Cr.', 'Status',
       'Balance (NPR)', 'Channel', 'Category'],
      dtype='object')


In [15]:
monthly = fin_df.groupby([pd.Grouper(freq = "M"), "Category"])["Dr."].sum().unstack(fill_value=0)

# print(monthly)

print(monthly.index[-1].to_period("M") + 1)

for cat in monthly.columns:
    if sum(monthly[cat]) == 0:
        monthly.drop([cat], axis = 1, inplace=True)

def summation(row):
    total_sum = 0
    for value in row:
        total_sum = total_sum + value
    return total_sum

monthly['Total'] = monthly.apply(lambda x: summation(x), axis = 1)

def percentage(row, col):
    percent = ((row[col] / row["Total"]) * 100)
    return percent

for col in monthly.columns:
    monthly[f"{col} %"] = monthly.apply(lambda x: percentage(x, col), axis = 1)



2025-07


C:\Users\swoye\AppData\Local\Temp\ipykernel_21616\3273054278.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = fin_df.groupby([pd.Grouper(freq = "M"), "Category"])["Dr."].sum().unstack(fill_value=0)


In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
warnings.filterwarnings('ignore')

In [9]:
monthly_income = 500000
target_savings = 90000

In [10]:
def check_stationary(timeseries, title):
    print(f"Result of Dickey-Fuller Test for {title}: ")
    dftest = adfuller(timeseries, autolag = "AIC")
    dfoutput = pd.Series(dftest[0:4], index = ["Test Statistics", "p-value", "#Lags Used", "Number of Obersvations Used"])
    for key, value in dftest[4].items():
        dfoutput['Critical value (%s)'%key] = value
    print(dfoutput)
    print("Is Stationary:", dftest[1] <= 0.05)
    return dftest[1] <= 0.05

In [11]:
def find_best_arima_model(data, max_p = 3, max_d = 3, max_q = 3):
    best_aic = np.inf
    best_order = None

    for p in range(max_p + 1):
        for d in range(max_d + 1):
            for q in range(max_q + 1):
                try:
                    arima_model = ARIMA(data, order = (p, d, q))
                    fit_model = arima_model.fit()
                    aic = fit_model.aic
    
                    if aic < best_aic:
                        best_aic = aic
                        best_order = (p, d, q)
                except:
                    continue
    
    return best_order, best_aic


In [12]:
def forecast_arima(data, order, steps = 1):
    arima_model = ARIMA(data, order=order)
    fitted_model = arima_model.fit()
    forecast = fitted_model.forecast(steps = steps)
    conf_int = fitted_model.get_forecast(steps = steps).conf_int()

    return forecast, conf_int, fitted_model

In [13]:
def create_budget_forecast(monthly_data, income, target_savings):
    forecasts = {}
    model_summary = {}

    cols = {col for col in monthly_data.columns if not col.endswith(" %") and col != "Total"}

    for col in cols:
        if col in monthly_data.columns:

            data = monthly_data[col].dropna()
            if(len(data) < 3):
                print(f"{col} has insufficient data for ARIMA modelling!!")
                forecasts[col] = data.mean() if len(data) > 0 else 0
                budget = None
                forecast = None
                model_summary = None

                return {}, model_summary, budget

            is_stationary = check_stationary(data, col)
    
            best_order, best_aic = find_best_arima_model(data)
    
            forecast, conf_int, fitted_model = forecast_arima(data, best_order)

            if (forecast.iloc[0] < 0):

                forecast = data.mean()
                forecasts[col] = forecast

                model_summary[col] = {
                'forecast': forecast,
                'aic': 0,
                'order': [0, 0, 0],
                'lower_bound': data.min(),
                'upper_bound': data.max()
                }

                print(f"{col}:")
                print(f"Forecast: {forecast}")
                print(f"Lower Bound: {data.min()}")
                print(f"Upper Bound: {data.max()}")

            else: 
                
                forecasts[col] = forecast.iloc[0]
        
                model_summary[col] = {
                'forecast': forecast.iloc[0],
                'aic': best_aic,
                'order': best_order,
                'lower_bound': conf_int.iloc[0, 0],
                'upper_bound': conf_int.iloc[0, 1]
                }

                print(f"{col}:")
                print(f"Best order: {best_order}")
                print(f"AIC: {best_aic}")
                print(f"Forecast: {forecast.iloc[0]}")
                print(f"Lower Bound: {conf_int.iloc[0, 0]}")
                print(f"Upper Bound: {conf_int.iloc[0, 1]}")

    total_forecast = sum(forecasts.values())
    budget = forecasts.copy()

    available_for_expenses = income - target_savings
    budget_surplus_deficit = available_for_expenses - total_forecast

    print(available_for_expenses)
    print(budget_surplus_deficit)

    if(budget_surplus_deficit > 0):
        print(f"Surplus: {budget_surplus_deficit}")
    elif(budget_surplus_deficit < 0):
        print(f"Deficit: {budget_surplus_deficit}")
    else:
        print("Balanced!!")

    cat_percent = {}

    for category, forecast_value in forecasts.items():
        percentage = (forecast_value / total_forecast) * 100
        cat_percent[category] = percentage
        print(f"{category:<20}: Rs. {forecast_value:>8,.2f} ({percentage:>5.1f}%)")

    if budget_surplus_deficit < 0:
        sorted_categories = sorted(forecasts.items(), key = lambda x:x[1], reverse=True)

        while budget_surplus_deficit < 0:
            made_progress = False  # Track if anything was deducted this round
        
            for i, (category, value) in enumerate(sorted_categories):
                current_value = budget.get(category, 0)
        
                if current_value <= 0:
                    continue  

            # for i, (category, value) in enumerate(sorted_categories):
            #     current_value = budget.get(category, 0)
        
            #     categ_percent = cat_percent[category]

            #     deduction_amount = (categ_percent/100) * value

            #     new_value = current_value - deduction_amount

            #     budget[category] = new_value

            #     budget_surplus_deficit += new_value

            # if budget_surplus_deficit >= -1e-2:
            #     break
        
                p = 50 / (i + 1)
                twt = (20 / 100) * current_value 
        
                if ((p / 100) * abs(budget_surplus_deficit)) < twt:
                    deduction_amount = (p / 100) * abs(budget_surplus_deficit)
                elif (((p / 2) / 100) * abs(budget_surplus_deficit)) < twt:
                    deduction_amount = ((p / 2) / 100) * abs(budget_surplus_deficit)
                elif (((p / 4) / 100) * abs(budget_surplus_deficit)) < twt:
                    deduction_amount = ((p / 4) / 100) * abs(budget_surplus_deficit)
                else:
                    deduction_amount = ((p / 8) / 100) * abs(budget_surplus_deficit)
        
                deduction_amount = min(deduction_amount, current_value)
        
                if deduction_amount <= 0:
                    continue 
        
                new_value = current_value - deduction_amount
                budget[category] = new_value
                budget_surplus_deficit += deduction_amount
                made_progress = True
        
                print(f"\nReducing {category}")
                print(f"Original: {current_value:.2f}")
                print(f"Deduction: {deduction_amount:.2f}")
                print(f"New Value: {new_value:.2f}")
                print(f"Updated Deficit: {budget_surplus_deficit:.2f}")
        
                if budget_surplus_deficit >= 0:
                    break
        
            if not made_progress:
                print("⚠️ No further deductions possible.")
                break
             
    return forecasts, model_summary, budget

In [14]:
def visualize_forecast(monthly_data, forecasts, model_summary):

    exp_cate = [col for col in monthly_data if not col.endswith(' %') and col != 'Total']

    n_col = 2
    cat_len = len(exp_cate)
    n_row = (cat_len + 1) // n_col + ((cat_len + 1) % n_col > 0)

    fig, axes = plt.subplots(n_row, n_col, figsize = (15, 5*n_row))
    axes = axes.flatten() if n_row > 1 else [axes] if n_col == 1 else axes

    for i, category in enumerate(exp_cate):
        ax = axes[i]

        monthly_data[category].plot(ax = ax, label = "Historical", marker = 'x')

        if category in forecasts:
            forecast_month = monthly_data.index[-1] + pd.DateOffset(month=1)
            ax.scatter(forecast_month, forecasts[category], color = 'red', s = 100, zorder = 5, label = 'Forecast')

            if category in model_summary:
                lower_bound = model_summary[category]['lower_bound']
                upper_bound = model_summary[category]['upper_bound']
                ax.fill_between([forecast_month, forecast_month], [lower_bound, upper_bound], alpha = 0.3, label = 'Confidence Interval')

        ax.set_title(f'{category}: Forecast')
        ax.set_ylabel('Amount')
        ax.legend()
        ax.grid(True, alpha = 0.3)

    plt.tight_layout()
    plt.show()

In [15]:
def analyze_trends(monthly_data):

    if 'Total' in monthly_data.columns:
        total_expense = monthly_data['Total']

    if len(total_expense) > 4:
        decomposition = seasonal_decompose(total_expense, model = 'additive', period=3)
        
        fig, axes = plt.subplots(4, 1, figsize = (12, 10))
        decomposition.observed.plot(ax = axes[0], title = 'Original')
        decomposition.trend.plot(ax = axes[1], title = 'trend')
        decomposition.seasonal.plot(ax = axes[2], title = 'seasonality')
        decomposition.resid.plot(ax = axes[3], title = 'Residual')
        plt.tight_layout()
        plt.show()

    if len(total_expense) >= 2:
            n_months = len(total_expense)

            growth_rate = ((total_expense.iloc[-1] / total_expense.iloc[0]) ** (1 / (n_months - 1))) - 1
     
            print(f"\nAverage Monthly Growth Rate: {growth_rate * 100:.2f}%")

In [19]:
forecasts, model_summary, budget = create_budget_forecast(monthly, monthly_income, target_savings)

print(budget)

# visualize_forecast(monthly, forecasts, model_summary)

# analyze_trends(monthly)

# budget_df = pd.DataFrame(budget.items(), columns=['Category', 'Forecast'])
# budget_df.to_csv('budget_forecast.csv', index=False)
# print('Forecast total', sum(forecasts.values()))
# print('Budget total', sum(budget.values()))
# print(f"\nBudget results saved to 'budget_forecast.csv'")

Result of Dickey-Fuller Test for Education: 
Test Statistics               -8.952345e+00
p-value                        8.655603e-15
#Lags Used                     0.000000e+00
Number of Obersvations Used    5.000000e+00
Critical value (1%)           -6.045114e+00
Critical value (5%)           -3.929280e+00
Critical value (10%)          -2.986810e+00
dtype: float64
Is Stationary: True
Education:
Best order: (1, 3, 1)
AIC: 65.555879988224
Forecast: 61042.76132340368
Lower Bound: 49851.881743122314
Upper Bound: 72233.64090368505
Result of Dickey-Fuller Test for Entertainment: 
Test Statistics               -3.936361
p-value                        0.001783
#Lags Used                     1.000000
Number of Obersvations Used    4.000000
Critical value (1%)           -7.355441
Critical value (5%)           -4.474365
Critical value (10%)          -3.126933
dtype: float64
Is Stationary: True
Entertainment:
Best order: (0, 3, 2)
AIC: 67.0329031196564
Forecast: 49149.093816918954
Lower Bound: 29

In [6]:
dict1 = {'Education': 55793.506523012235, 'Entertainment': 46983.776211757475, 'Travel': 59359.346969681435, 'Subscriptions': 33605.4784581175, 'Banking & Finance': 51273.82805630554, 'Groceries & Shopping': 53353.4662297945, 'Dining & Food': 53606.38655088269, 'Personal Care': 56024.21100044864, 'Housing & Utilities': 0.0}
dict2 = {'Education': 55793.506523012235, 'Entertainment': 46983.776211757475, 'Travel': 59359.346969681435, 'Subscriptions': 33605.4784581175, 'Banking & Finance': 51273.82805630554, 'Groceries & Shopping': 53353.4662297945, 'Dining & Food': 53606.38655088269, 'Personal Care': 56024.21100044864, 'Housing & Utilities': 0.0}
new_dict = {'Category': [], 'B_Amount': [], 'F_Amount': []}

for key, value in dict1.items():
    new_dict['Category'].append(key)
    new_dict['B_Amount'].append(value)

for values in dict2.values():
    new_dict['F_Amount'].append(values)

print(new_dict)

df = pd.DataFrame(new_dict)
print(df)

{'Category': ['Education', 'Entertainment', 'Travel', 'Subscriptions', 'Banking & Finance', 'Groceries & Shopping', 'Dining & Food', 'Personal Care', 'Housing & Utilities'], 'B_Amount': [55793.506523012235, 46983.776211757475, 59359.346969681435, 33605.4784581175, 51273.82805630554, 53353.4662297945, 53606.38655088269, 56024.21100044864, 0.0], 'F_Amount': [55793.506523012235, 46983.776211757475, 59359.346969681435, 33605.4784581175, 51273.82805630554, 53353.4662297945, 53606.38655088269, 56024.21100044864, 0.0]}
               Category      B_Amount      F_Amount
0             Education  55793.506523  55793.506523
1         Entertainment  46983.776212  46983.776212
2                Travel  59359.346970  59359.346970
3         Subscriptions  33605.478458  33605.478458
4     Banking & Finance  51273.828056  51273.828056
5  Groceries & Shopping  53353.466230  53353.466230
6         Dining & Food  53606.386551  53606.386551
7         Personal Care  56024.211000  56024.211000
8   Housing & 